In [1]:
dataset_name = "cifar10"
model_type = "cnn" 
nump_epochs = 40
pth = "./trajectories"

In [2]:
!pip install ipywidgets

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F



img_size =  4
ch_size = 3
# Define the ConvNet architecture
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(ch_size, 8, kernel_size=3, padding=1)  # 1 input channel, 8 output channels
        self.conv2 = nn.Conv2d(8, 16, kernel_size=3, padding=1) # 8 input channels, 16 output channels
        self.conv3 = nn.Conv2d(16, 32, kernel_size=3, padding=1) # 8 input channels, 16 output channels
        self.fc1 = nn.Linear(32 * img_size * img_size, 120)   # Fully connected layer with 120 neurons
        self.fc2 = nn.Linear(120, 10)           # Output layer with 10 neurons (10 classes)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = F.relu(F.max_pool2d(self.conv3(x), 2))
        x = x.view(-1, 32 * img_size * img_size)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)



True


In [4]:
class EarlyStopping:
    def __init__(self, patience=5, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss > self.best_loss - self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.counter = 0

# Instantiate early stopping
early_stopping = EarlyStopping(patience=3, min_delta=0.01)

In [5]:
if dataset_name == "mnist":
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])
    train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
    test_dataset = datasets.MNIST(root='./data', train=False, transform=transform)
else:
    transform = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
    transform2 = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
    train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)                              
    test_dataset = datasets.CIFAR10(root='./data', train=False, transform=transform2)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)


# Define the training loop
def train(epoch, model, optimizer):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data.cuda())
        loss = F.nll_loss(output, target.cuda())
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

# Define the test loop
def test(epoch, model, optimizer):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data.cuda())
            test_loss += F.nll_loss(output, target.cuda(), reduction='sum').item()  # Sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # Get the index of the max log-probability
            correct += pred.eq(target.cuda().view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({100. * correct / len(test_loader.dataset):.0f}%)\n')
    return test_loss


Files already downloaded and verified


In [6]:
import os


# Initialize the model and optimizer
model = ConvNet()
model=model.cuda()
optimizer = optim.SGD(model.parameters(), lr= 0.001, momentum=0.9, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,  'min',factor=0.5, patience=2)


    
    
def save_model(epoch, model):
    # Save the model after each epoch
    save_path = f'{pth}/models_{model_type}_{dataset_name}/model_epoch_{epoch}.pt'
    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    torch.save(model.state_dict(), save_path)
    print(f'Model saved to {save_path}')
    
# Train and test the model with early stopping
save_model(0, model)
for epoch in range(1, nump_epochs + 1):
    train(epoch, model, optimizer)
    val_loss = test(epoch, model, optimizer)
    scheduler.step(val_loss)
    early_stopping(val_loss)    
    save_model(epoch, model)
    if early_stopping.early_stop:
        print("Early stopping triggered")
        break
    



Model saved to /home/elhamod/projects/AE/trajectories/models_cnn_cifar10/model_epoch_0.pt
Train Epoch: 1 [0/50000 (0%)]	Loss: 2.305030
Train Epoch: 1 [3200/50000 (6%)]	Loss: 2.314895
Train Epoch: 1 [6400/50000 (13%)]	Loss: 2.291991
Train Epoch: 1 [9600/50000 (19%)]	Loss: 2.300772
Train Epoch: 1 [12800/50000 (26%)]	Loss: 2.296607
Train Epoch: 1 [16000/50000 (32%)]	Loss: 2.288496
Train Epoch: 1 [19200/50000 (38%)]	Loss: 2.290217
Train Epoch: 1 [22400/50000 (45%)]	Loss: 2.271148
Train Epoch: 1 [25600/50000 (51%)]	Loss: 2.253256
Train Epoch: 1 [28800/50000 (58%)]	Loss: 2.183405
Train Epoch: 1 [32000/50000 (64%)]	Loss: 2.186069
Train Epoch: 1 [35200/50000 (70%)]	Loss: 2.059025
Train Epoch: 1 [38400/50000 (77%)]	Loss: 1.894166
Train Epoch: 1 [41600/50000 (83%)]	Loss: 2.131043
Train Epoch: 1 [44800/50000 (90%)]	Loss: 1.986991
Train Epoch: 1 [48000/50000 (96%)]	Loss: 2.095779

Test set: Average loss: 1.9820, Accuracy: 3016/10000 (30%)

Model saved to /home/elhamod/projects/AE/trajectories/mode

Train Epoch: 9 [38400/50000 (77%)]	Loss: 1.237029
Train Epoch: 9 [41600/50000 (83%)]	Loss: 1.134897
Train Epoch: 9 [44800/50000 (90%)]	Loss: 0.928643
Train Epoch: 9 [48000/50000 (96%)]	Loss: 1.357773

Test set: Average loss: 1.1298, Accuracy: 5988/10000 (60%)

Model saved to /home/elhamod/projects/AE/trajectories/models_cnn_cifar10/model_epoch_9.pt
Train Epoch: 10 [0/50000 (0%)]	Loss: 0.978076
Train Epoch: 10 [3200/50000 (6%)]	Loss: 1.180147
Train Epoch: 10 [6400/50000 (13%)]	Loss: 1.261152
Train Epoch: 10 [9600/50000 (19%)]	Loss: 1.082274
Train Epoch: 10 [12800/50000 (26%)]	Loss: 1.139479
Train Epoch: 10 [16000/50000 (32%)]	Loss: 0.980664
Train Epoch: 10 [19200/50000 (38%)]	Loss: 0.981477
Train Epoch: 10 [22400/50000 (45%)]	Loss: 0.767608
Train Epoch: 10 [25600/50000 (51%)]	Loss: 1.146775
Train Epoch: 10 [28800/50000 (58%)]	Loss: 1.022586
Train Epoch: 10 [32000/50000 (64%)]	Loss: 0.821219
Train Epoch: 10 [35200/50000 (70%)]	Loss: 0.896489
Train Epoch: 10 [38400/50000 (77%)]	Loss: 1.04

Train Epoch: 18 [12800/50000 (26%)]	Loss: 0.906812
Train Epoch: 18 [16000/50000 (32%)]	Loss: 0.839918
Train Epoch: 18 [19200/50000 (38%)]	Loss: 1.108831
Train Epoch: 18 [22400/50000 (45%)]	Loss: 0.840091
Train Epoch: 18 [25600/50000 (51%)]	Loss: 0.767514
Train Epoch: 18 [28800/50000 (58%)]	Loss: 0.919496
Train Epoch: 18 [32000/50000 (64%)]	Loss: 1.068704
Train Epoch: 18 [35200/50000 (70%)]	Loss: 0.860219
Train Epoch: 18 [38400/50000 (77%)]	Loss: 0.807061
Train Epoch: 18 [41600/50000 (83%)]	Loss: 1.058735
Train Epoch: 18 [44800/50000 (90%)]	Loss: 0.954662
Train Epoch: 18 [48000/50000 (96%)]	Loss: 1.126605

Test set: Average loss: 0.8969, Accuracy: 6857/10000 (69%)

Model saved to /home/elhamod/projects/AE/trajectories/models_cnn_cifar10/model_epoch_18.pt
Train Epoch: 19 [0/50000 (0%)]	Loss: 0.656199
Train Epoch: 19 [3200/50000 (6%)]	Loss: 0.565618
Train Epoch: 19 [6400/50000 (13%)]	Loss: 1.070232
Train Epoch: 19 [9600/50000 (19%)]	Loss: 1.003807
Train Epoch: 19 [12800/50000 (26%)]	Loss:

Train Epoch: 26 [48000/50000 (96%)]	Loss: 0.591271

Test set: Average loss: 0.8014, Accuracy: 7221/10000 (72%)

Model saved to /home/elhamod/projects/AE/trajectories/models_cnn_cifar10/model_epoch_26.pt
Train Epoch: 27 [0/50000 (0%)]	Loss: 0.829713
Train Epoch: 27 [3200/50000 (6%)]	Loss: 0.739181
Train Epoch: 27 [6400/50000 (13%)]	Loss: 0.668941
Train Epoch: 27 [9600/50000 (19%)]	Loss: 0.550705
Train Epoch: 27 [12800/50000 (26%)]	Loss: 0.768478
Train Epoch: 27 [16000/50000 (32%)]	Loss: 1.068703
Train Epoch: 27 [19200/50000 (38%)]	Loss: 0.922552
Train Epoch: 27 [22400/50000 (45%)]	Loss: 0.790703
Train Epoch: 27 [25600/50000 (51%)]	Loss: 0.670758
Train Epoch: 27 [28800/50000 (58%)]	Loss: 0.784539
Train Epoch: 27 [32000/50000 (64%)]	Loss: 0.723523
Train Epoch: 27 [35200/50000 (70%)]	Loss: 1.095324
Train Epoch: 27 [38400/50000 (77%)]	Loss: 0.738453
Train Epoch: 27 [41600/50000 (83%)]	Loss: 0.856452
Train Epoch: 27 [44800/50000 (90%)]	Loss: 0.639209
Train Epoch: 27 [48000/50000 (96%)]	Loss:

In [7]:
from torchsummary import summary

img_dim  =4 *8
summary(model, input_size=(3, img_dim, img_dim))  # Input size should match the input dimensions of your model


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 32, 32]             224
            Conv2d-2           [-1, 16, 16, 16]           1,168
            Conv2d-3             [-1, 32, 8, 8]           4,640
            Linear-4                  [-1, 120]          61,560
            Linear-5                   [-1, 10]           1,210
Total params: 68,802
Trainable params: 68,802
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 0.11
Params size (MB): 0.26
Estimated Total Size (MB): 0.38
----------------------------------------------------------------
